# DataStore - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **DataStore** w celu zapisania niewielkich ilości danych w pliku.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />

Jest to biblioteka, która pomaga w przechowywaniu i zarządzaniu danymi w sposób bardziej zoptymalizowany i bezpieczny niż starsze rozwiązania, takie jak `SharedPreferences` (zostanie omówiony w kolejnym przykładzie).

`DataStore` jest często wykorzystywany do przechowywania niewielkich ilości danych, takich jak preferencje użytkownika, ustawienia aplikacji, stan sesji i tym podobne. Nie jest on jednak odpowiedni do przechowywania dużej ilości danych, jakie można znaleźć w bazach danych.

Oferuje dwa główne rodzaje implementacji:
- `Preferences DataStore` - Ten rodzaj jest podobny do `SharedPreferences`, ale jest oparty na protokole `Kotlin Coroutines` i zapewnia bezpieczne przechowywanie danych. Można w nim przechowywać dane w postaci *klucz-wartość*, gdzie klucze są ciągami znaków, a wartości mogą być różnymi typami danych, takimi jak liczby, ciągi znaków itp.
- `Proto DataStore` - Ten rodzaj pozwala na zapisywanie danych w formacie `protobuf`. Jest to format serializacji danych opracowany przez firmę Google.

Główne cehy korzystania z `DataStore` to:
- Wsparcie dla asynchronicznych operacji dzięki `Kotlin Coroutines`.
- Bezpieczeństwo wątkowe - unika problemów synchronizacyjnych, które mogą występować w przypadku `SharedPreferences`.
- Automatyczne obsługiwania zmian wartości, co pozwala na łatwe reagowanie na zmiany danych.
- Możliwość zapisywania i odczytywania niestandardowych typów danych dzięki `Protobuf`.

W przykładowej aplikacji wykorzystamy `DataStore` do zapisania i odczytania nazwy użytkownika.

Dodajmy niezbędne zależności do projektu:

do bloku `dependencies`
```kotlin
    implementation (libs.androidx.lifecycle.viewmodel.ktx)
    implementation (libs.androidx.fragment.ktx)
    implementation (libs.androidx.datastore.preferences)
```

do pliku `libs.versions.toml`

```kotlin
[versions]
agp = "8.5.2"
datastorePreferences = "1.1.1"
fragmentKtx = "1.8.2"
kotlin = "1.9.0"
coreKtx = "1.13.1"
junit = "4.13.2"
junitVersion = "1.2.1"
espressoCore = "3.6.1"
appcompat = "1.7.0"
lifecycleViewmodelKtx = "2.8.4"
material = "1.12.0"
activity = "1.9.1"
constraintlayout = "2.1.4"

[libraries]
androidx-core-ktx = { group = "androidx.core", name = "core-ktx", version.ref = "coreKtx" }
androidx-datastore-preferences = { module = "androidx.datastore:datastore-preferences", version.ref = "datastorePreferences" }
androidx-fragment-ktx = { module = "androidx.fragment:fragment-ktx", version.ref = "fragmentKtx" }
androidx-lifecycle-viewmodel-ktx = { module = "androidx.lifecycle:lifecycle-viewmodel-ktx", version.ref = "lifecycleViewmodelKtx" }
junit = { group = "junit", name = "junit", version.ref = "junit" }
androidx-junit = { group = "androidx.test.ext", name = "junit", version.ref = "junitVersion" }
androidx-espresso-core = { group = "androidx.test.espresso", name = "espresso-core", version.ref = "espressoCore" }
androidx-appcompat = { group = "androidx.appcompat", name = "appcompat", version.ref = "appcompat" }
material = { group = "com.google.android.material", name = "material", version.ref = "material" }
androidx-activity = { group = "androidx.activity", name = "activity", version.ref = "activity" }
androidx-constraintlayout = { group = "androidx.constraintlayout", name = "constraintlayout", version.ref = "constraintlayout" }

[plugins]
android-application = { id = "com.android.application", version.ref = "agp" }
jetbrains-kotlin-android = { id = "org.jetbrains.kotlin.android", version.ref = "kotlin" }


```

Chcemy dodawać nazwy użytkowników, podobnie jak w poprzednich przykładach dodajmy *dummydata* w celu ułatwienia generacji przykładowych nazw.

In [ ]:
object DataProvider {

    private val usernames = listOf(
        "CoolDude123",
        "SuperStar",
        "GamerGirl99",
        "TechMaster",
        "MusicLover",
        "FitnessFreak",
        "Traveler_21",
        "FoodieQueen",
        "NatureLover",
        "Fashionista",
        "Bookworm42",
        "MovieBuff",
        "AdventureSeeker",
        "PetLover_87",
        "SportsFanatic",
        "ArtisticSoul",
        "StarGazer",
        "YogaEnthusiast",
        "PhotographyPro",
        "DreamChaser",
        "BeachBum123",
        "CoffeeAddict",
        "GameChanger",
        "LaughOutLoud",
        "MindfulSoul",
        "HappyGoLucky",
        "TechGeek1",
        "FoodExplorer",
        "FitnessJunkie"
    )

    val username: String
        get() = usernames.random()
}

Dodajmy klasę pomocniczą `SaveUsernameDataStore`, która służy do przechowywania i pobierania nazwy użytkownika przy użyciu `DataStore`. Wykorzystamy wzorzec Singleton, ponieważ chcemy posiadać tylko jeden obiekt przez który będziemy komunikowac się z `DataStore`.

In [ ]:
object SaveUsernameDataStore {
    private val Context.dataStore: DataStore<Preferences> by preferencesDataStore("user_prefs")
    private val USERNAME_KEY = stringPreferencesKey("USERNAME")

    suspend fun storeUsername(context: Context, username: String) {
        context.dataStore.edit { preferences ->
            preferences[USERNAME_KEY] = username
        }
    }

    fun getUsernameFlow(context: Context): Flow<String> {
        return context.dataStore.data.map { preferences ->
            preferences[USERNAME_KEY] ?: ""
        }
    }
}

- `private val Context.dataStore: DataStore<Preferences> by preferencesDataStore("user_prefs")` - Ta linia definiuje rozszerzenie (extension property) dla klasy `Context`. Tworzy ono obiekt `DataStore` typu `Preferences` o nazwie `user_prefs`. Ten `DataStore` będzie używany do przechowywania i pobierania preferencji użytkownika.
- `private val USERNAME_KEY = stringPreferencesKey("USERNAME")` - Ta linia definiuje prywatną stałą, która reprezentuje klucz używany do przechowywania i pobierania nazwy użytkownika w `DataStore`. Jest to obiekt typu `Preferences.Key<String>`.
- `suspend fun storeUsername(context: Context, username: String)` Ta funkcja służy do zapisywania nazwy użytkownika w `DataStore`.
- `fun getUsernameFlow(context: Context): Flow<String>` - Ta funkcja zwraca strumień asynchroniczny typu `Flow<String>`, który będzie emitować nazwę użytkownika za każdym razem, gdy wartość ulegnie zmianie.

Dodajmy repozytorium

In [ ]:
class UserRepository(private val application: Application) {
    fun getUsername() = SaveUsernameDataStore.getUsernameFlow(application)
    suspend fun add(username: String) = SaveUsernameDataStore.storeUsername(application, username)
    suspend fun clear() = SaveUsernameDataStore.storeUsername(application, "")
}

Podobnie jak przy wykorzystaniu `Room`, musimy przekazać kontekst aplikacji jako parametr konstruktora. Więc musimy przekazać go również przez `AndroidViewModel`.

Dodajmy `Viewmodel` - w kodzie nie ma żadnych nowych elementów.

In [ ]:
class UserViewModel(application: Application) : AndroidViewModel(application) {
    private val repository: UserRepository = UserRepository(application)
    private val _username = MutableStateFlow("")
    val username: StateFlow<String>
        get() = _username

    init {
        fetchUser()
    }

    private fun fetchUser() {
        viewModelScope.launch {
            repository.getUsername().collect { username ->
                _username.value = username
            }
        }
    }

    fun addUsername(username: String) {
        viewModelScope.launch {
            repository.add(username)
        }
    }

    fun clearUsername(){
        viewModelScope.launch {
            repository.clear()
        }
    }
}

Następnie dodajmy Fragment.

In [ ]:
class UserFragment : Fragment() {

     private lateinit var binding: FragmentUserBinding

    private val viewModel: UserViewModel by viewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentUserBinding.inflate(inflater)

        viewLifecycleOwner.lifecycleScope.launch {
            viewModel.username.collectLatest { user ->
                binding.textView.text = "Current saved user: $user"
            }
        }

        binding.addButton.setOnClickListener { viewModel.addUsername(DataProvider.username) }
        binding.clearButton.setOnClickListener { viewModel.clearUsername() }

        return binding.root
    }
}

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMHZ0Mmt6cjJwYjVrYzE0NWdpbHNqYXM5ZjVvNzc1bmhqZm1zY2F5NSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/kd4nprniMKNt9fBZIv/giphy.gif" width="200" />